In [66]:
import os
import plotly
import subprocess
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import collections
import re
from operator import itemgetter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

plotly.tools.set_credentials_file(username='Fernet', api_key='zC7ogVfVT5UPcpu2VKse')
plotly.offline.init_notebook_mode(connected=True)

#---------------- Settings ----------------

image_ext = "svg" #Extension para bajar los graficos
poda = 5 #Cuantos datos se dejan a cada lado al podar
n = 175  #El n fijo para el grafico de tiempo en función de calidad de solucion
#image_h = 1000
#image_w = 1400
image_h = 600
image_w = 800

#---------------- Funciones ----------------

def read_source_data(input_file, source_name):
    data = pd.read_csv(input_file, sep=";")
    symbols = data["Symbol"].tolist()
    probabilities = data["Probability"].tolist()
    information = data["Information"].tolist()
    entropy = data["Entropy"][0]
    max_entropy = data["Maximum Entropy"][0]
    source_data = SourceData(Symbols=symbols, Probabilities=probabilities,
                        Information=information, Entropy=entropy, Max_Entropy=max_entropy, Source_Name=source_name)
    return source_data

def plot_symbols_info(source_data, fig_title):
    
    symbols = source_data.Symbols
    print len(symbols)
    information = source_data.Information
    entropy = source_data.Entropy
    max_entropy = source_data.Max_Entropy

    symbols = symbols + ["Entropía", "Entropía Máxima "]
    information = information + [entropy, max_entropy] 
    x_title = "Símbolos"
    y_title = "Información"
    fig = create_bar_fig(symbols, information, x_title, y_title, fig_title)
    
    plotly.offline.iplot(fig)

def plot_symbols_info_V2(sources, fig_title):
    
    traces = [go.Bar(x = source_data.Symbols, y = source_data.Information,name = source_data.Source_Name)
              for source_data in sources]
    
    x_title = "Símbolos"
    y_title = "Información"
    layout = go.Layout(
                title = fig_title,
                xaxis = dict(title = x_title),
                yaxis = dict(title = y_title)
    )
    
    fig = go.Figure(data=traces, layout=layout)
    
    plotly.offline.iplot(fig)
    
def plot_broadcast_info(source_data, fig_title):
    symbols = source_data.Symbols
    probabilities = source_data.Probabilities
    
    broadcast_probability = 0
    for i in range(len(symbols)):
        s = symbols[i]
        if 'broadcast' in s:
            broadcast_probability += probabilities[i]
    labels = ['Broadcast', 'Unicast']
    values = [broadcast_probability, 1 - broadcast_probability]
    layout = go.Layout(title=fig_title)
    #Create pie fig
    fig = {
        "data": [{"values": values, "labels": labels, "type":"pie"}],
        "layout": {"title": fig_title}
    }
    
    plotly.offline.iplot(fig)
       
    
def plot_probabilities_info(source_data, fig_title):
    symbols = source_data.Symbols
    probabilities = source_data.Probabilities
    fig = create_bar_fig(symbols, probabilities, fig_title)
    plotly.offline.iplot(fig)



def tiempo_vs_calidad(data_path, algoritmo, tam):
    data = pd.read_csv(data_path, sep = ";")
    df_algoritmo = data.loc[data["Algoritmo"] == algoritmo] #Tiene todas las filas del algoritmo pedido
    df_algoritmo = df_algoritmo.loc[df_algoritmo["V"] == tam] #Tiene todas las filas para el V pedido
    df_algoritmo["Calidad_solucion"] = df_algoritmo["Solucion"] / (df_algoritmo["Solucion"] + df_algoritmo["Dif_frontera"])
    
    tiempos = df_algoritmo["Tiempo"]
    calidades = df_algoritmo["Calidad_solucion"]
    return tiempos, calidades


def create_bar_fig(x_values, y_values, x_title, y_title, fig_title="Bar fig"):
    data = [go.Bar(
                x = x_values,
                y = y_values
        )]

    layout = go.Layout(
        xaxis=dict(
            title = x_title
        ),
        yaxis=dict(
            title = y_title
        ),
        title = fig_title
    )
    fig = go.Figure(data=data, layout=layout)
    return fig

#Toma un vector de tuplas (x, y, nombre) y devuelve una figura con cada tupla de datos
def crear_fig(traces, xtitle, ytitle, fig_title, marcadores=None):
    data = []
    for trace in traces:
        #Para que todas las traces sean distintas ademas del color
        i = 0
        if not marcadores:
            modo = str(i)
        else:
            modo = marcadores
        i += 1

        new_trace = go.Scatter(
            x = trace[0],
            y = trace[1],
            name = trace[2],
            mode = modo
                
        )
        data.append(new_trace)
    
    layout = go.Layout(
        xaxis=dict(
            tickmode = "auto",
            #nticks = cant_exp,
            ticks = "inside",
            #tick0 = 0,
            #ticklen = 8,
            #dtick = 3,
            title = xtitle,
            tickangle = 0
        ),
        yaxis=dict(
            tick0 = 0,
            zeroline = "False",
            ticks='inside',
            ticklen=8,
            tickwidth=3,
            tickcolor='#000',
            title = ytitle
        ),
        legend=dict(
            x=10,
            y=1,
            traceorder='normal',
            font=dict(
                family='sans-serif',
                size=14,
                color='#000'
            ),
            bgcolor='#E2E2E2',
            bordercolor='#FFFFFF',
            borderwidth=2
        ),
        title = fig_title
    )
    
    return go.Figure(data=data, layout=layout)

#---------------- Script ----------------

#---Data gathering---

SourceData = collections.namedtuple('SourceData', ['Symbols', "Probabilities",
                                    "Information", "Entropy", "Max_Entropy", "Source_Name"])

#--S1--

#Home network 1
source_data_hn1_S1 = read_source_data("homeNetwork1_S1.csv", "Red Hogar 1, S1")

#Home network 2
#source_data_hn2_S1 = read_source_data("homeNetwork2_S1.csv", "Red Hogar 2, S1")

#Work network
#source_data_wn_S1 = read_source_data("workNetwork_S1.csv", "Red Trabajo, S1")

#Lab network
source_data_ln_S1 = read_source_data("labNetwork_S1.csv", "Red Laboratorio, S1")


#--S2--

#Home network 1
source_data_hn1_S2 = read_source_data("homeNetwork1_S2.csv", "Red Hogar 1, S2")

#Home network 2
#source_data_hn2_S2 = read_source_data("homeNetwork2_S2.csv", "Red Hogar 2, S2")

#Work network
#source_data_wn_S2 = read_source_data("workNetwork_S2.csv", "Red Trabajo, S2")

#Lab network
source_data_ln_S2 = read_source_data("labNetwork_S2.csv", "Red Laboratorio, S2")


#Graficos

def call_on_sources(function, source_type):
    if function == plot_symbols_info:
        plot_fun_title = "Gráfico de información de cada símbolo. Red: "
    elif function == plot_broadcast_info:
        plot_fun_title = "Gráfico de tráfico de broadcast sobre el total. Red: "
    elif function == plot_probabilities_info:
        plot_fun_title = "Gráfico de probabilidades de cada símbolo. Red: "
    elif function == plot_ARP_network:
        plot_fun_title = "Gráfico de la red ARP subyacente. Red: "
    else:
        print "Error in call_on_sources source_type parameter. Must be 1 or 2"
    
    if source_type == 1:
        function(source_data_hn1_S1, plot_fun_title + "hogar 1, fuente tipo S1")
        function(source_data_hn2_S1, plot_fun_title + "hogar 2, fuente tipo S1")
        #function(source_data_wn_S1, plot_fun_title + "trabajo, fuente tipo S1")
        #function(source_data_ln_S1, plot_fun_title + "laboratorio, fuente tipo S1")
    elif source_type == 2:
        function(source_data_hn1_S2, plot_fun_title + "hogar 1, fuente tipo S2")
        #function(source_data_hn2_S2, plot_fun_title + "hogar 2, fuente tipo S2")
        #function(source_data_wn_S2, plot_fun_title + "trabajo, fuente tipo S2")
        function(source_data_ln_S2, plot_fun_title + "laboratorio, fuente tipo S2")

def plot_graphs_S1():
    #call_on_sources(plot_symbols_info, 1)
    #call_on_sources(plot_broadcast_info, 1)
    #call_on_sources(plot_probabilities_info, 1)
    networks = [source_data_hn1_S1, source_data_ln_S1]
    plot_symbols_info_V2(networks, "Comparación redes de hogar vs labo")
    
    
def plot_graphs_S2():
    call_on_sources(plot_symbols_info, 2)
    
    
def plot_all_graphs():
    plot_graphs_S1()
    plot_graphs_S2()


plot_all_graphs()




#plotly.offline.iplot(fig)
#plotly.offline.iplot(fig, image = image_ext, image_width = image_w, image_height = image_h, filename = "Ejemplo")




8


189


In [52]:
import os
import plotly
import subprocess
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import collections
import re
from operator import itemgetter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

plotly.tools.set_credentials_file(username='Fernet', api_key='zC7ogVfVT5UPcpu2VKse')
plotly.offline.init_notebook_mode(connected=True)

#---------------- Settings ----------------

image_ext = "svg" #Extension para bajar los graficos
poda = 5 #Cuantos datos se dejan a cada lado al podar
n = 175  #El n fijo para el grafico de tiempo en función de calidad de solucion
#image_h = 1000
#image_w = 1400
image_h = 600
image_w = 800

#---------------- Pruebas ----------------

